### Chroma Setup and Chunk Loading
Sets up persistant client and loads previously computed chunks

In [4]:
from pathlib import Path
import chromadb

# Using the same path as previous notebooks
Relative_Database_path = "./chroma_Data"
Absolute_Database_path = Path(Relative_Database_path).resolve()

# Initialize the persistent client
client = chromadb.PersistentClient(path=Absolute_Database_path)
print(f"[INFO] ChromaDB client initialized at: {Absolute_Database_path}")

# List existing collections
existing_collections = client.list_collections()
print(f"Existing collections: {[c.name for c in existing_collections]}")

[INFO] ChromaDB client initialized at: C:\Users\micro\Desktop\Abhinav college\Resources\Sem 7\Advanced NLP\RAG_for_research_papers\VectorDB\chroma_Data
Existing collections: ['my_collection']


In [3]:
import pickle
from langchain.schema import Document
# No need for fitz or RecursiveCharacterTextSplitter here, as we are loading from a file.

file_path = "../Chunking/harry_potter_chunks.pkl"

loaded_docs = []

try:
    with open(file_path, "rb") as f: # 'rb' mode for reading in binary
        loaded_docs = pickle.load(f)
    print(f"Successfully loaded {len(loaded_docs)} chunks from '{file_path}'.")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"Error loading file: {e}")

# Now you can inspect the loaded documents to verify.
print("\nHere is the metadata of a loaded chunk:")
if loaded_docs:
    print(loaded_docs[0].metadata)

Successfully loaded 621 chunks from '../Chunking/harry_potter_chunks.pkl'.

Here is the metadata of a loaded chunk:
{'source': '../harrypotter.pdf', 'page_number': 1}


### Set up Embedding Function
Will use default SentenceTransformer for generating embeddings

In [5]:
# Install if needed
# !pip install sentence_transformers

# Set up embedding function
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
print("Embedding function initialized with model: all-MiniLM-L6-v2")

Embedding function initialized with model: all-MiniLM-L6-v2


### Creating new Collection

In [6]:
from datetime import datetime

# Create a new collection with a unique name
collection_name = "harry_potter_collection"

# Get or create the collection
collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=embedding_function,
    metadata={
        "description": "Harry Potter book chunks",
        "created": str(datetime.now())
    }
)

print(f"Collection '{collection_name}' created or accessed successfully")

Collection 'harry_potter_collection' created or accessed successfully


### Add data to collection
The chunks have to be given an id and added to the collection now

In [7]:
import uuid

# Prepare documents for ChromaDB
ids = []
documents = []
metadatas = []

# Process each loaded document chunk
for i, doc in enumerate(loaded_docs):
    # Generate a unique ID (you could use a more deterministic approach if needed)
    doc_id = f"hp_chunk_{i}"
    
    # Get the document text
    document_text = doc.page_content
    
    # Get the document metadata
    metadata = doc.metadata
    
    # Add to our lists
    ids.append(doc_id)
    documents.append(document_text)
    metadatas.append(metadata)

# Add documents in batches to avoid memory issues
batch_size = 500
total_added = 0

for i in range(0, len(ids), batch_size):
    end_idx = min(i + batch_size, len(ids))
    
    collection.add(
        ids=ids[i:end_idx],
        documents=documents[i:end_idx],
        metadatas=metadatas[i:end_idx]
    )
    
    total_added += end_idx - i
    print(f"Added batch: {i} to {end_idx-1} ({end_idx-i} items)")

print(f"Successfully added {total_added} documents to collection '{collection_name}'")

Added batch: 0 to 499 (500 items)
Added batch: 500 to 620 (121 items)
Successfully added 621 documents to collection 'harry_potter_collection'


In [8]:
# Check collection count
count = collection.count()
print(f"Total documents in collection: {count}")

# Peek at the first few entries
peek = collection.peek(limit=3)
print("\nSample entries:")
for i, (doc_id, doc_text, metadata) in enumerate(zip(
    peek['ids'], peek['documents'], peek['metadatas']
)):
    print(f"\n--- Document {i+1} ---")
    print(f"ID: {doc_id}")
    print(f"Text: {doc_text[:100]}...")
    print(f"Metadata: {metadata}")

Total documents in collection: 621

Sample entries:

--- Document 1 ---
ID: hp_chunk_0
Text: M
 
CHAPTER  ONE
THE BOY WHO LIVED
r. and Mrs. Dursley, of number four, Privet Drive, were proud to ...
Metadata: {'page_number': 1, 'source': '../harrypotter.pdf'}

--- Document 2 ---
ID: hp_chunk_1
Text: their greatest fear was that somebody would discover it. They didn’t think
they could bear it if any...
Metadata: {'page_number': 1, 'source': '../harrypotter.pdf'}

--- Document 3 ---
ID: hp_chunk_2
Text: When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday our story
starts, there was nothing abou...
Metadata: {'source': '../harrypotter.pdf', 'page_number': 2}


### Querying the Database

In [9]:
# Rich table for displaying results (optional but nice)
try:
    from rich.console import Console
    from rich.table import Table
    
    console = Console()
    use_rich = True
except ImportError:
    use_rich = False
    print("Rich package not found. Using standard print.")

# Function to display query results
def print_results(results, use_rich=use_rich):
    if use_rich:
        table = Table(show_header=True, header_style="bold magenta")
        table.add_column("Rank", width=6)
        table.add_column("Document ID")
        table.add_column("Document Text", width=60)
        table.add_column("Page")
        table.add_column("Distance")
        
        docs = results['documents'][0]
        ids = results['ids'][0]
        metas = results['metadatas'][0]
        distances = results['distances'][0]
        
        for i, (doc, doc_id, meta, dist) in enumerate(zip(docs, ids, metas, distances)):
            table.add_row(
                str(i+1),
                doc_id,
                (doc[:100] + "...") if len(doc) > 100 else doc,
                str(meta.get('page_number', 'N/A')),
                f"{dist:.4f}"
            )
        
        console.print(table)
    else:
        # Standard print version
        for i, (doc, meta, dist) in enumerate(zip(
            results['documents'][0], 
            results['metadatas'][0], 
            results['distances'][0]
        )):
            print(f"\n--- Result {i+1} ---")
            print(f"Text: {doc[:100]}...")
            print(f"Metadata: {meta}")
            print(f"Distance: {dist:.4f}")

# Run a query
query = "What happened when Harry first visited Ollivanders wand shop?"

results = collection.query(
    query_texts=[query],
    n_results=3,
    include=["documents", "metadatas", "distances"]
)

print(f"\nResults for query: '{query}'")
print_results(results)


Results for query: 'What happened when Harry first visited Ollivanders wand shop?'


┏━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━┓
┃ Rank   ┃ Document ID  ┃ Document Text                                                ┃ Page ┃ Distance ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━┩
│ 1      │ hp_chunk_169 │ useful, carry yer mail an’ everythin’.”                      │ 68   │ 0.3300   │
│        │              │ Twenty minutes later, they left Eeylops Owl Emporium, which  │      │          │
│        │              │ ...                                                          │      │          │
│ 2      │ hp_chunk_175 │ Harry took the wand and (feeling foolish) waved it around a  │ 70   │ 0.3934   │
│        │              │ bit, but Mr.                                                 │      │          │
│        │              │ Ollivander snatched it out ...                               │      │          │
│ 3      │ hp_chunk_177 │ Ollivander cried, “Oh, bravo! Yes, indeed, oh, very good.    │ 71   │ 0.4126   │
│        │              │ Well, well, well . . .                                       │      │          │
│        │              │ how curious . . . h...                                       │      │          │
└────────┴──────────────┴──────────────────────────────────────────────────────────────┴──────┴──────────┘